In [ ]:
# add project root (parent of figures) to module search path
import os
os.chdir(os.path.abspath(".."))  # now CWD is ai_scientist_project/
print("Current working directory:", os.getcwd())
#  import packages
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
def extract_program_data_new(census, reference_loss=22.02000046):
    """
    Extracts program data from the NEW census arrays. These are not stored in such a stupid way as the old ones.
    Each program is a row in a long numpy array with the following columns:
    - 0: iteration number
    - 1: island number
    - 2: batch number
    - 3: LLM name
    - 4: loss
    - 5: time in seconds
    - 6: parent1 id (int)
    - 7: parent2 id (int)
    - 8: y_eval (float)
     
    Returns a dictionary with the following keys
    - 'scores': np.ndarray of scores (exp(-(loss - reference_loss)))
    - 'losses': np.ndarray of losses
    - 'running_max_scores': np.ndarray of running max scores
    - 'times': np.ndarray of times in seconds
    - 'parent1_id': np.ndarray of parent1 ids (int)
    - 'parent2_id': np.ndarray of parent2 ids (int)
    - 'program_id': list of tuples representing the program ids in format (iteration, island
    , batch)
    - 'llm_name': np.ndarray of LLM names (str)
    - 'innovation_indices': list of indices where innovations occurred (where the max score increases)
    - 'generations': list of lists, where generations[k] is a list of the kth order ancestors of the winner.
                     So generations[0] = [winner_index], generations[1] = [parent1_index, parent2_index], etc.
    - 'y_eval': np.ndarray of y_eval values (float)
    """
    # extract data from the census
    n_programs = census.shape[0]  # number of programs
    losses = census[:, 4]  # losses
    times = census[:, 5]  # times in seconds
    program_tuple_id = [tuple(census[i, :3]) for i in range(n_programs)]  # program ids as tuples (iteration, island, batch)
    parent1_tuple_id = census[:, 6]  # parent1 ids
    parent2_tuple_id = census[:, 7]  # parent2 ids
    llm_name = census[:, 3]  # LLM names
    y_eval = census[:, 8]  # y_eval values
    n_free_params = census[:, 9]  # number of free parameters (not used in this function, but could be useful later)

    # now loop through all programs with index >= 2 and find their parents
    parent1_int_id = [-1, -1]  # initialize parent ids with -1
    parent2_int_id = [-1, -1]  # initialize parent ids with -1
    for i in range(2, n_programs):
        p1_tuple_id = parent1_tuple_id[i]
        p2_tuple_id = parent2_tuple_id[i]
        # print(f"Processing program {i}: {program_tuple_id[i]}, parents: {p1_tuple_id}, {p2_tuple_id}")
        # print(f"p1_tuple_id.dtype: {type(p1_tuple_id)}, p2_tuple_id.dtype: {type(p2_tuple_id)}")
        p1_int_id = program_tuple_id.index(p1_tuple_id)
        p2_int_id = program_tuple_id.index(p2_tuple_id)
        # print(f"p1_int_id: {p1_int_id}, p2_int_id: {p2_int_id}")
        parent1_int_id.append(p1_int_id)
        parent2_int_id.append(p2_int_id)
    # check everyone has 2 parents
    assert len(parent1_int_id) == n_programs, "parent1_int_id must have the same length as census"
    assert len(parent2_int_id) == n_programs, "parent2_int_id must have the same length as census"

    # convert to numpy arrays
    parent1_id = np.array(parent1_int_id, dtype=int)
    parent2_id = np.array(parent2_int_id, dtype=int)
    llm_name = np.array(llm_name, dtype=str)
    # subtract the start time from all times
    times = times - np.min(times)
    # get scores
    losses = np.array(losses, dtype=float)
    losses_relative = losses - reference_loss  # relative losses
    scores = np.exp(-losses_relative)  # convert losses to scores
    running_max_scores = np.maximum.accumulate(scores)  # running max scores
    # look for innovations (where the max score increases) and get the indices
    innovation_threshold = 0.01  # threshold for innovation detection
    innovation_indices = np.where(np.diff(running_max_scores) > innovation_threshold)[0]
    innovation_indices = innovation_indices.tolist()  # convert to list for easier manipulation
    innovation_indices = [1] + innovation_indices  # include the first index

    # build family tree of winner
    index = np.argmax(scores)
    generations = [[index]]
    # now go through the most recent generation and find the parents
    all_seed = False
    while not all_seed:
        current_generation = generations[-1]
        all_seed = all([i < 2 for i in current_generation])  # check if all are seed programs
        parent_generation = []
        for idx in current_generation:
            p1_idx, p2_idx = parent1_id[idx], parent2_id[idx]
            if p1_idx >= 0:
                parent_generation.append(p1_idx)
            if p2_idx >= 0:
                parent_generation.append(p2_idx)
        parent_generation = list(set(parent_generation))
        if len(parent_generation) == 0:
            break
        generations.append(parent_generation)

    results_dict = {
        'scores': scores,
        'losses': losses,
        'running_max_scores': running_max_scores,
        'times': times,
        'parent1_id': parent1_id,
        'parent2_id': parent2_id,
        'program_id': program_tuple_id,
        'llm_name': llm_name,
        'innovation_indices': innovation_indices,
        'generations': generations,
        'y_eval': y_eval,
        'n_free_params': n_free_params
    }
    return results_dict

In [ ]:
census_paths = [# 'program_databases/07-15/16-22-07 (big_only)/combined/census.npy',
                # 'program_databases/07-15/17-05-19 (big_only)/combined/census.npy',
                # 'program_databases/07-15/17-48-16 (big_only)/combined/census.npy',
                # 'program_databases/07-15/18-31-33 (big_only)/combined/census.npy',

                # 'program_databases/07-16/00-59-22 (mix)/combined/census.npy',
                # 'program_databases/07-16/01-46-55 (mix)/combined/census.npy',
                # 'program_databases/07-16/02-29-34 (mix)/combined/census.npy',
                # 'program_databases/07-16/03-15-03 (mix)/combined/census.npy',

                # 'program_databases/07-17/07-14-37 (mix + image)/combined/census.npy',
                # 'program_databases/07-17/08-10-17 (mix + image)/combined/census.npy',
                # 'program_databases/07-17/09-07-26 (mix + image)/combined/census.npy',
                # 'program_databases/07-17/10-05-55 (mix + image)/combined/census.npy',

                # 'program_databases/07-20/01-37-40 (image feedback)/combined/census.npy',
                # 'program_databases/07-20/02-43-00 (image feedback)/combined/census.npy',
                # 'program_databases/07-20/03-45-10 (image feedback)/combined/census.npy',
                # 'program_databases/07-20/04-47-23 (image feedback)/combined/census.npy',

                # 'program_databases/07-16/12-46-54 (big + no_image)/combined/census.npy',
                # 'program_databases/07-16/13-18-43 (big + no_image)/combined/census.npy',
                # 'program_databases/07-16/13-51-51 (big + no_image)/combined/census.npy',
                # 'program_databases/07-16/14-23-46 (big + no_image)/combined/census.npy',

                # 'program_databases/07-16/10-46-45 (mix + no_image)/combined/census.npy',
                # 'program_databases/07-16/11-18-03 (mix + no_image)/combined/census.npy',
                # 'program_databases/07-16/11-48-51 (mix + no_image)/combined/census.npy',
                # 'program_databases/07-16/12-18-10 (mix + no_image)/combined/census.npy',

                # 'program_databases/07-17/11-02-49 (mix + no_image)/combined/census.npy',
                # 'program_databases/07-17/11-42-11 (mix + no_image)/combined/census.npy',
                # 'program_databases/07-17/12-22-36 (mix + no_image)/combined/census.npy',
                # 'program_databases/07-17/14-55-12 (mix + no_image)/combined/census.npy',

                # 'program_databases/07-17/04-58-28 (no_param_est)/combined/census.npy',
                # 'program_databases/07-17/05-31-58 (no_param_est)/combined/census.npy',
                # 'program_databases/07-17/06-05-20 (no_param_est)/combined/census.npy',
                # 'program_databases/07-17/06-40-16 (no_param_est)/combined/census.npy',

                # 'program_databases/07-17/19-02-25 (no_gradient)/combined/census.npy',
                # 'program_databases/07-17/20-10-14 (no_gradient)/combined/census.npy',
                # 'program_databases/07-17/21-11-44 (no_gradient)/combined/census.npy',
                # 'program_databases/07-17/22-13-56 (no_gradient)/combined/census.npy',

                # 'program_databases/07-20/05-52-26 (no image)/combined/census.npy',
                # 'program_databases/07-20/06-26-59 (no image)/combined/census.npy',
                # 'program_databases/07-20/07-01-03 (no image)/combined/census.npy',
                # 'program_databases/07-20/07-35-59 (no image)/combined/census.npy',

                # 'program_databases/07-21/15-31-23 (no_image)/combined/census.npy',
                # 'program_databases/07-21/18-30-33 (no_image)/combined/census.npy',
                # 'program_databases/07-21/19-33-36 (no_image)/combined/census.npy',
                # 'program_databases/07-21/20-34-39 (no_image)/combined/census.npy',

                # 'program_databases/07-22/02-35-30 (no image)/combined/census.npy',
                # 'program_databases/07-22/03-16-56 (no image)/combined/census.npy',
                # 'program_databases/07-22/04-01-16 (no image)/combined/census.npy',
                # 'program_databases/07-22/04-47-09 (no image)/combined/census.npy',

                # 'program_databases/07-22/05-31-26 (image)/combined/census.npy',
                # 'program_databases/07-22/06-16-02 (image)/combined/census.npy',
                # 'program_databases/07-22/07-00-37 (image)/combined/census.npy',
                # 'program_databases/07-22/07-45-07 (image)/combined/census.npy',
                # 'program_databases/07-23/01-00-00 (image)/combined/census.npy',

                # 'program_databases/07-23/03-10-12 (text)/combined/census.npy',
                # 'program_databases/07-23/04-09-16 (text)/combined/census.npy',
                # 'program_databases/07-23/05-08-08 (text)/combined/census.npy',
                # 'program_databases/07-23/06-07-12 (text)/combined/census.npy',

                # 'program_databases/07-22/23-18-55 (image)/combined/census.npy',
                # 'program_databases/07-23/00-21-26 (image)/combined/census.npy',
                # 'program_databases/07-23/01-16-39 (image)/combined/census.npy',
                # 'program_databases/07-23/02-14-04 (image)/combined/census.npy',

                'program_databases/07-30/15-16-32/combined/census.npy',
                'program_databases/07-30/16-11-02/combined/census.npy',
                'program_databases/07-30/17-01-31/combined/census.npy',

                'program_databases/07-30/17-52-59/combined/census.npy',
                'program_databases/07-30/18-44-20/combined/census.npy',
                'program_databases/07-30/19-31-37/combined/census.npy',

                # 'program_databases/07-31/03-14-29/combined/census.npy',
                # 'program_databases/07-31/03-52-27/combined/census.npy',
                # 'program_databases/07-31/04-31-51/combined/census.npy',
                # 'program_databases/07-31/05-08-24/combined/census.npy',

                # 'program_databases/07-31/05-49-09/combined/census.npy',
                # 'program_databases/07-31/06-20-58/combined/census.npy',
                # 'program_databases/07-31/06-55-12/combined/census.npy',
                # 'program_databases/07-31/07-37-03/combined/census.npy',

                ]
results_list = []
for i, path in enumerate(census_paths):
    census = np.load(path, allow_pickle=True)
    print(f"Processing {path}...")
    results = extract_program_data_new(census,reference_loss=28.85)
    results_list.append(results)

times = [results['times'] for results in results_list]
scores = [results['scores'] for results in results_list]
running_max_scores = [results['running_max_scores'] for results in results_list]
generations = [np.array([results['program_id'][i][0] for i in range(len(results['program_id']))]) for results in results_list]

# # calculate losses at sampled times
# t_max = max([np.max(t) for t in times])  # find the maximum time across all runs
# t_samples = np.linspace(0, t_max, n_time_samples, endpoint=False)
# # for each t_sample and each run, find the closest time and get the max score at that time
# max_scores_sampled = np.zeros((len(running_max_scores), len(t_samples)))
# for i, running_max_score in enumerate(running_max_scores):
#     for j, t_sample in enumerate(t_samples):
#         # find the closest time to t_sample
#         closest_time_idx = np.argmin(np.abs(times[i] - t_sample))
#         max_scores_sampled[i, j] = running_max_score[closest_time_idx]

# # calculate mean and std for each group
# mean_scores = []
# std_scores = []
# for i, group in enumerate(group_indices):
#     group_scores = max_scores_sampled[group, :]
#     mean_scores.append(np.mean(group_scores, axis=0))
#     std_scores.append(np.std(group_scores, axis=0)) 

In [ ]:
simulation_no = 4
y_eval = results_list[simulation_no]['y_eval']
scores = results_list[simulation_no]['scores']
y_eval = np.array([np.array(y_eval[i]) for i in range(len(y_eval))])
y_eval = y_eval.reshape(y_eval.shape[0], -1)  # flatten the last two dimensions
# nan -> num
y_eval = np.nan_to_num(y_eval, nan=0.0)  # replace NaN with 0
# look at the first few cells
y_eval = y_eval[:, :100 * 20]

colours = ['blue', 'orange', 'green', 'red']
program_indices = [0, 1]
score_argmax = np.argmax(results_list[simulation_no]['scores'])
program_indices += [score_argmax]

plt.figure(figsize=(12, 6))
n_cells_view = 3  # number of cells to view
for i, program_index in enumerate(program_indices):
    plt.plot(y_eval[program_index, :n_cells_view * 100], 
             label=f'Program {program_index + 1}', 
             color=colours[i], 
             linewidth=2.0, 
             alpha=0.8)
# add dashed vertical lines at the start of each cell
for i in range(1, n_cells_view + 1):
    plt.axvline(x=i * 100, color='grey', linestyle='--', linewidth=0.5, alpha=0.5)
plt.xlabel('theta')
plt.ylabel('prediction')
plt.show()

In [ ]:
import umap
# perform nonlinear dimensionality reduction on the y_eval data
umap_model = umap.UMAP(n_components=2, n_neighbors=10, min_dist=1, metric='euclidean')
y_eval_embedded = umap_model.fit_transform(y_eval)
# from sklearn.manifold import TSNE
# tsne_model = TSNE(n_components=2, random_state=42, perplexity=50)
# y_eval_embedded = tsne_model.fit_transform(y_eval)


In [ ]:
# Plot an image of the t-SNE embedding, by interpolating the points
# center so both dims have mean 0
y_eval_embedded = y_eval_embedded - np.mean(y_eval_embedded, axis=0)
# scale so min and max are -1 and 1
y_eval_embedded = y_eval_embedded / np.max(np.abs(y_eval_embedded), axis=0)
sigma = 0.07
x = np.linspace(-1.0, 0.75, 100)  # x-axis range for the t-SNE embedding
y = np.linspace(-1.0, 1.0, 100)  # y-axis range for the t-SNE embedding
X, Y = np.meshgrid(x, y)
Z = np.zeros(X.shape)
D = np.zeros(X.shape)  # Denominator for normalization
const = 1e-2

for i in range(len(y_eval_embedded)):
    xi = y_eval_embedded[i, 0]
    yi = y_eval_embedded[i, 1]
    zi = scores[i]
    # Interpolate the score onto the grid
    Z += zi * np.exp(-((X - xi) ** 2 + (Y - yi) ** 2) / (2 * sigma ** 2))
    D += const + np.exp(-((X - xi) ** 2 + (Y - yi) ** 2) / (2 * sigma ** 2))

Z = Z / D  # Normalize by the denominator to get the average score at each point

surf = go.Surface(x=X, y=Y, z=Z,
                  colorscale='Viridis', showscale=False,
                  name='Smoothed surface')

# ---------- helper that returns Z at any (x, y) -------------------
Ny, Nx = Z.shape
dx, dy = x[1] - x[0], y[1] - y[0]
ix_scale, iy_scale = 1 / dx, 1 / dy

def surface_z(xp, yp):
    ix = np.clip(((xp - x[0]) * ix_scale).round().astype(int), 0, Nx - 1)
    iy = np.clip(((yp - y[0]) * iy_scale).round().astype(int), 0, Ny - 1)
    return Z[iy, ix]                # note: row (y) index first!

# ---------- 1. Ancestors of the best program ----------------------------
best_program_index = np.argmax(scores)
generations = results_list[simulation_no]['generations']
ancestor_indices = [ancestor for generation in generations for ancestor in generation]

z_ancestors = surface_z(y_eval_embedded[ancestor_indices, 0],  # ancestors' smoothed scores
                        y_eval_embedded[ancestor_indices, 1]) + 0.02

all_pts = go.Scatter3d(
    x=y_eval_embedded[ancestor_indices, 0],
    y=y_eval_embedded[ancestor_indices, 1],
    z=z_ancestors,
    mode='markers',
    marker=dict(size=4,
                # color=scores[ancestor_indices],  # colour matches surface height
                color='black',  # all black
                symbol='circle',
                # colorscale='Viridis',
                opacity=0.9,
                showscale=False),
    name='Ancestors of Best Program'
)


# ---------- 2. Seed programmes (just two special points) ----------
seed_idx = [0, 1]
z_seed   = surface_z(y_eval_embedded[seed_idx, 0],
                     y_eval_embedded[seed_idx, 1]) + 0.02

seed_pts = go.Scatter3d(
    x=y_eval_embedded[seed_idx, 0],
    y=y_eval_embedded[seed_idx, 1],
    z=z_seed,
    mode='markers',
    marker=dict(size=8, color='red', symbol='cross'),
    name='Seed programmes'
)

# ---------- 3. Assemble and show ----------------------------------
fig = go.Figure(data=[surf, all_pts, seed_pts])
for i in range(len(ancestor_indices) - 1):
    p1, p2 = ancestor_indices[i], ancestor_indices[i + 1]
    if p1 < 0 or p2 < 0:
        continue                       # skip undefined parents

    fig.add_trace(go.Scatter3d(
        x=y_eval_embedded[[p1, p2], 0],
        y=y_eval_embedded[[p1, p2], 1],
        z=z_ancestors[[i,   i+1]],
        mode='lines',
        line=dict(color='black', width=1.5, dash='dash'),
        showlegend=False
    ))
    
fig.update_layout(
    width = 1000,        # px
    height = 700,        # px
    scene = dict(
        xaxis_title='t‑SNE‑1',
        yaxis_title='t‑SNE‑2',
        zaxis_title='Smoothed score',
        camera=dict(eye=dict(x=1.3, y=1.3, z=0.9))
    ),
    margin=dict(l=0, r=0, t=40, b=0)
)
fig.show()